This notebook reports on the test results for test `grain_size_strain_pinned_state.prm` and allows to compare the ASPECT test against analytical results. Over time the model maintains nearly constant temperatures, pressures and strain rates and only utilizes dislocation creep. The grain size in the test approaches the equilibrium grain size for t -> infinity. For this simple case we can compute the analytical equilibrium grain size and compare it to the one in the ASPECT testcase.

In [ ]:
import numpy as np

In [ ]:
# define constants here
q                       = 4.
grain_growth_prefactor  = 1.6e-22
grain_growth_activation = 39072
f1                      = 1e-6
phase_distribution      = 0.6*0.4

hg         = (2/np.pi)*(2/np.pi)
R_times_T  = 8.314*1600

In steady state, we can compute equilibrium grain size in the pinned state using Eq. 8 of Mulyukova and Bercovici (2018) by assuming the grain size change rate $\dot{R} = 0$: 

\begin{equation}
\dot{R} = 0 = \frac {3 \phi_1 \phi_2 k_g \sqrt h_g^{-q}}{q R^{q-1}} \exp(\frac{-E_g}{R_gT})  - \frac{f \sqrt h_g \Psi}{3 \phi_1 \phi_2} R^2
\end{equation}

Here, $\phi_1 \phi_2$ is the phase distribution function and $\sqrt{h_g} = 2/\pi$. The shear heating term $\Psi$ is $2\dot\varepsilon\tau$, where $\tau = 2 \eta \dot\varepsilon$, therefore, $\Psi = 4 \eta \dot\varepsilon ^2$

We will use

\begin{align}
B_{\text{growth}} = \frac {3 \phi_1\phi_2 k_g \sqrt {h_g}^{-q}}{q} \exp(\frac{-E_g}{R_gT}) \\
B_{\text{reduction}} = - \frac{f \sqrt{h_g} \Psi}{3\phi_1\phi_2}
\end{align}

to combine the the prefactors of R (grain_size) such that the above equation can be written as:

\begin{equation}
0 = \frac{B_{\text{growth}}}{R^{(q-1)}} - B_{\text{reduction}}*R^2,
\end{equation}

and rearranging the above equation yields:

\begin{equation}
R = \left(\frac{B_{\text{growth}}}{B_{\text{reduction}}}\right)^{\frac{1}{q+1}}.
\end{equation}


In [ ]:
def equilibrium_grain_size (viscosity, q):
    
    shear_heating = 4*strain_rate*strain_rate*viscosity
    
    grain_reduction_term = f1*np.sqrt(hg)*shear_heating/(3*phase_distribution)
    
    grain_growth_term    = (hg**(-q/2) * 3 * phase_distribution * grain_growth_prefactor * \
                                 np.exp(-grain_growth_activation/R_times_T))/q
    
    return (grain_growth_term/grain_reduction_term)**(1/(q+1))

As shown above the equilibrium grain size depends on the viscosity through the shear heating term $\Psi$. Because the viscosity of diffusion creep in turn depends on the grain size this relationship makes the equilibrium grain size hard to compute analytically. To simplify the system we disable diffusion creep and only utilize dislocation creep for this test case, with the dislocation viscosity computed as:

\begin{equation}
\eta = \frac{1}{2} A^{\frac{-1}{n}} \dot\varepsilon^{1/n - 1} \exp{\frac{-E_d}{nR_gT}}.
\end{equation}

Here, $A$ is the dislocation prefactor, $n$ the stress exponent, $\dot{\epsilon}$ the strain rate, $E_d$ the dislocation activation energy, $R_g$ the universal gas constant, and $T$ temperature.

In [ ]:
# In our test case of simple shear, we apply vx = y/10^5 per year and vy = 0. Therefore, we only have the yx 
# component of the strain rate tensor = 1/2*(vx/y) or 1/2*(1/10^5) per year.

strain_rate            = (1e-5/(3600*24*365.25))/2
dislocation_prefactor  = 1e-13
dislocation_activation = 390720
stress_exponent        = 3

dislocation_viscosity = (dislocation_prefactor)**(-1/stress_exponent)*strain_rate**(1/stress_exponent - 1) \
                        *np.exp(dislocation_activation/(stress_exponent*R_times_T))

Assuming the parameters above, which are also used as input parameters for the ASPECT test `grain_size_strain_pinned_state.prm`, we can calculate the expected equilibrium grain size:

In [ ]:
analytical_equilibrium_grain_size = equilibrium_grain_size (dislocation_viscosity, 4)
print ("Analytical equilibrium grain size: ", analytical_equilibrium_grain_size)


And the actual ASPECT test value after 4e8 years (when it is converged):

In [ ]:
aspect_equilibrium_grain_size = 0.0151242683
print ("ASPECT equilibrium grain size: ", aspect_equilibrium_grain_size)
relative_equilibrium_grain_size_error = np.abs(aspect_equilibrium_grain_size - analytical_equilibrium_grain_size) / analytical_equilibrium_grain_size
print ("Relative error: ", relative_equilibrium_grain_size_error)